<a href="https://colab.research.google.com/github/deitar/file-monitoring-auto-git/blob/main/File_Monitor_Auto_Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import random
import string
import os
import time
import subprocess
from datetime import datetime

In [36]:
# Create random dummy files for experimentation
def generate_random_text(length):
    return ''.join(random.choice(string.ascii_letters + string.digits + string.punctuation) for _ in range(length))

def create_random_files(file_count, file_length):
    for i in range(file_count):
        file_name = f'file{i+1}.txt'
        random_text = generate_random_text(file_length)
        with open(file_name, 'w') as file:
            file.write(random_text)
        print(f'File "{file_name}" created.')

# Usage example
file_count = 5
file_length = 100

create_random_files(file_count, file_length)

File "file1.txt" created.
File "file2.txt" created.
File "file3.txt" created.
File "file4.txt" created.
File "file5.txt" created.


In [ ]:
GIT_USERNAME = ''

In [ ]:
GIT_PASSWORD = ''

In [49]:
FOLDER_TO_LOCAL_GIT_REPO = '.' # set path to git local repository

In [50]:
FOLDER_PATH_TO_MONITOR = None # None if you are monitoring current directory

In [51]:
MONITOR_TIMEOUT_SECS = 2

In [52]:
os.chdir(FOLDER_TO_LOCAL_GIT_REPO) # set working directory to airflow

In [58]:
def generate_current_datetime_str():
  current_datetime = datetime.now()
  current_datetime_string = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
  return current_datetime_string

In [53]:
def run_git_command(command):
    try:
        output = subprocess.check_output(['git'] + command.split(), stderr=subprocess.STDOUT)
        return output.decode('utf-8').strip()
    except subprocess.CalledProcessError as e:
        error_message = e.output.decode('utf-8').strip()
        print(f'Git command failed with error: {error_message}')

# Example usage
command = 'checkout -b new_branch'
result = run_git_command(command)
print(result)


Git command failed with error: fatal: not a git repository (or any of the parent directories): .git
None


In [56]:
def git_push_with_credentials(remote_name='origin', branch_name='chore'):
    try:
        push_command = f'git push {remote_name} {branch_name}'
        process = subprocess.Popen(push_command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, error = process.communicate()

        if process.returncode == 0:
            print("Git push successful.")
        else:
            print("Git push failed.")
            if b'Username for' in error:
                username = GIT_USERNAME # input("Enter your username: ")
                password = GIT_PASSWORD # input("Enter your password: ")
                credentials = f'{username}:{password}'
                process = subprocess.Popen(push_command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                           stdin=subprocess.PIPE)
                output, error = process.communicate(input=credentials.encode())

                if process.returncode == 0:
                    print("Git push successful.")
                else:
                    print("Git push failed with provided credentials.")
            else:
                print("An error occurred during the Git push.")
                print("Error message:", error.decode('utf-8').strip())
    except OSError as e:
        print("Error executing Git push command:", e)

# # Example usage
# git_push_with_credentials()


Git push failed.
An error occurred during the Git push.
Error message: fatal: not a git repository (or any of the parent directories): .git


In [ ]:
file_times = {} # dictionary keeping the filepaths and last modified time as value

while True:
  for filename in os.listdir(FOLDER_PATH_TO_MONITOR):
    file_path = os.path.join(FOLDER_PATH_TO_MONITOR or '', filename)
    if os.path.isfile(file_path): # check if path corresponds to a file
      new_modified_file_time = os.path.getmtime(file_path)
      old_modified_file_time = file_times.get(file_path)
      if old_modified_file_time is None or (old_modified_file_time is not None and new_modified_file_time > old_modified_file_time):
        # if file has not been captured when program starts
        # or file has been modified
        # push to git

        # set branch name
        filename_without_ext = os.path.splitext(filename)[0]
        git_branch_name = f'modified_{filename_without_ext.replace(" ","_")}'

        print(file_path)
        print(new_modified_file_time)
        # git checkout -b branch
        run_git_command(f'checkout -b {git_branch_name}')
        # git add filepath
        run_git_command(f'add {file_path}')
        # git commit -m "modified {filename}"
        run_git_command(f'commit -m "modified {filename} - {generate_current_datetime_str()}"')
        # git push
        git_push_with_credentials(branch_name = git_branch_name)

      file_times[file_path] = new_modified_file_time
    # print(file_times)

    time.sleep(MONITOR_TIMEOUT_SECS)